# Import Libraries

In [30]:
import numpy as np
import pandas as pd
import plotly.express as px

import sys
sys.path.append('../')
from utils.data import *
import re

* Get target Match

In [31]:
df_filtered_match = df_matches_with_clubs_names[df_matches_with_clubs_names['match_id'] == 59183].copy()
df_filtered_match

,match_id,home_team_id,away_team_id,home_team_goals,away_team_goals,match_date,matchweek,referee,stadium_id,season,club_id_x,home_team_name,club_id_y,away_team_name,stadium_name,city
6799,59183,163,158,2,1,2021-04-21,32,David Coote,176,2020/21,163,Tottenham Hotspur,158,Southampton,Tottenham Hotspur Stadium,London


* Get target match stats

In [32]:
df_filtered_match_stats = df_club_stats[df_club_stats['match_id'] == df_filtered_match['match_id'].values[0]].copy()
df_filtered_match_stats = df_filtered_match_stats.merge(df_club, on='club_id')
df_filtered_match_stats

,match_id,club_id,possession,shots,shots_on_target,touches,passes,tackles,clearances,corners,offsides,fouls_conceded,yellow_cards,red_cards,stadium_id,club_name
0,59183,163,61.7,12,3,833,610,18,13,5,2,10,2,0,176,Tottenham Hotspur
1,59183,158,38.3,10,5,583,371,29,16,5,0,14,1,0,171,Southampton


# Timeline

* Get timeline in target match

In [33]:
filtered_match_timeline_mask = (df_player_performance['match_id'] == df_filtered_match['match_id'].values[0]) & (df_player_performance['type_of_stat'].str.lower() != 'assist')
df_filtered_match_timeline_info = df_player_performance[filtered_match_timeline_mask]
df_filtered_match_timeline_info

,player_performance_id,player_id,match_id,type_of_stat,minute
48189,48195,p36903,59183,goal,60
48190,48196,p85971,59183,goal (by penalty),90
48191,48197,p84939,59183,goal,30


* Get scorer(s) info

In [34]:
df_filtered_match_timeline_info = df_filtered_match_timeline_info.merge(df_player_stats, on=['match_id', 'player_id'])
df_filtered_match_timeline_info

,player_performance_id,player_id,match_id,type_of_stat,minute,is_in_starting_11,substitution_on,substitution_off,yellow_card,red_card,is_home_side
0,48195,p36903,59183,goal,60,1,NaN,83',0,0,1
1,48196,p85971,59183,goal (by penalty),90,1,NaN,NaN,0,0,1
2,48197,p84939,59183,goal,30,1,NaN,57',0,0,0


* Get name(s) of scorer(s)

In [35]:
df_filtered_match_timeline_info = df_filtered_match_timeline_info.merge(df_player, on='player_id')
df_filtered_match_timeline_info

,player_performance_id,player_id,match_id,type_of_stat,minute,is_in_starting_11,substitution_on,substitution_off,yellow_card,red_card,is_home_side,player_name,country
0,48195,p36903,59183,goal,60,1,NaN,83',0,0,1,Gareth Bale,Wales
1,48196,p85971,59183,goal (by penalty),90,1,NaN,NaN,0,0,1,Son Heung-Min,South Korea
2,48197,p84939,59183,goal,30,1,NaN,57',0,0,0,Danny Ings,England


* Get teams' of scorer(s) 

In [36]:
home_team_name = df_filtered_match['home_team_name'].values[0]
away_team_name = df_filtered_match['away_team_name'].values[0]

In [37]:
df_filtered_match_timeline_info['club_name'] = df_filtered_match_timeline_info['is_home_side'].apply(lambda x: home_team_name if x == 1 else away_team_name)

# get interesting columns only
df_filtered_match_timeline_info = df_filtered_match_timeline_info[['player_name', 'type_of_stat', 'minute', 'club_name']]

* Casting `minute` dtype from `object` to `int`

In [38]:
df_filtered_match_timeline_info['minute'] = df_filtered_match_timeline_info['minute'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]) if len(x.split('+'))-1 else int(x))
df_filtered_match_timeline_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   player_name   3 non-null      object
 1   type_of_stat  3 non-null      object
 2   minute        3 non-null      int64 
 3   club_name     3 non-null      object
dtypes: int64(1), object(3)
memory usage: 120.0+ bytes


/tmp/ipykernel_15498/1954843430.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



* Sort by `minute`

In [39]:
df_filtered_match_timeline_info = df_filtered_match_timeline_info.sort_values(by='minute').reset_index(drop=True)

In [40]:
df_filtered_match_timeline_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   player_name   3 non-null      object
 1   type_of_stat  3 non-null      object
 2   minute        3 non-null      int64 
 3   club_name     3 non-null      object
dtypes: int64(1), object(3)
memory usage: 224.0+ bytes


* Convert the minute column to start and end times

In [41]:
df_filtered_match_timeline_info['start_time'] = pd.to_datetime(df_filtered_match['match_date'].values[0]) + pd.to_timedelta(df_filtered_match_timeline_info['minute'], unit='m')
df_filtered_match_timeline_info['end_time'] = df_filtered_match_timeline_info['start_time'] + pd.to_timedelta('1 minute')

* Uppercase for events for better visualiztion

In [42]:
df_filtered_match_timeline_info['type_of_stat'] = df_filtered_match_timeline_info['type_of_stat'].str.upper()
df_filtered_match_timeline_info

,player_name,type_of_stat,minute,club_name,start_time,end_time
0,Danny Ings,GOAL,30,Southampton,2021-04-21 00:30:00,2021-04-21 00:31:00
1,Gareth Bale,GOAL,60,Tottenham Hotspur,2021-04-21 01:00:00,2021-04-21 01:01:00
2,Son Heung-Min,GOAL (BY PENALTY),90,Tottenham Hotspur,2021-04-21 01:30:00,2021-04-21 01:31:00


In [59]:
# color map of each event in match
color_map = {'GOAL': 'darkgreen', 
             'GOAL (BY PENALTY)': 'green',
             'SECOND YELLOW CARD (AND/OR RED': 'lightgray',
             'OWN GOAL': 'gray',
             'RED CARD': 'red'}


# Create the Gantt chart
timeline_figure = px.timeline(df_filtered_match_timeline_info, 
                              x_start='start_time', 
                              x_end='end_time', 
                              y='club_name', 
                              color='type_of_stat',
                              color_discrete_map=color_map,
                              labels={'type_of_stat': 'Event', 'club_name': 'Club'},
                              hover_name='player_name',
                              hover_data={'minute': ':.0f', 'start_time': False, 'end_time': False, })

# Add annotations for each timeline
for index, row in df_filtered_match_timeline_info.iterrows():
    home_team_name = df_filtered_match['home_team_name'].values[0]
    away_team_name = df_filtered_match['away_team_name'].values[0]

    home_team_goals_sum_mask = (df_filtered_match_timeline_info['club_name'] == home_team_name) & (df_filtered_match_timeline_info['type_of_stat'].str.lower().isin(['goal', 'goal (by penalty)']))
    home_team_own_goals_sum_mask = (df_filtered_match_timeline_info['club_name'] == home_team_name) & (df_filtered_match_timeline_info['type_of_stat'].str.lower() == 'own goal')

    away_team_goals_sum_mask = (df_filtered_match_timeline_info['club_name'] == away_team_name) & (df_filtered_match_timeline_info['type_of_stat'].str.lower().isin(['goal', 'goal (by penalty)'])) 
    away_team_own_goals_sum_mask = (df_filtered_match_timeline_info['club_name'] == away_team_name) & (df_filtered_match_timeline_info['type_of_stat'].str.lower() == 'own goal')

    home_team_goals_sum = len(df_filtered_match_timeline_info[:index+1][home_team_goals_sum_mask | away_team_own_goals_sum_mask])
    away_team_goals_sum = len(df_filtered_match_timeline_info[:index+1][away_team_goals_sum_mask | home_team_own_goals_sum_mask])

    timeline_figure.add_annotation(x=row['start_time'], y=row['club_name'], 
                        text=str(home_team_goals_sum) + ': ' + str(away_team_goals_sum) ,
                        textangle=-45, font=dict(color='white'), bgcolor='darkgreen', 
                        showarrow=False, xanchor='center',yshift=74)

# Set the chart title and axis labels
timeline_figure.update_layout(title='Timeline',
                   xaxis_title='Time',
                   yaxis_title='Team')

timeline_figure.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

# Show the chart
timeline_figure.show()

/tmp/ipykernel_15498/3633157335.py:31: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_15498/3633157335.py:32: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [44]:
df_player_performance['type_of_stat'].str.upper().unique().tolist()

['GOAL',
 'ASSIST',
 'GOAL (BY PENALTY)',
 'SECOND YELLOW CARD (AND/OR RED',
 'OWN GOAL',
 'RED CARD']

# Possession

In [45]:
df_filtered_match_possession = df_filtered_match_stats[['club_name', 'possession']].copy()
df_filtered_match_possession

,club_name,possession
0,Tottenham Hotspur,61.7
1,Southampton,38.3


In [46]:
possession_figure = px.pie(df_filtered_match_possession, values='possession', hole=0.5,
                                          names='club_name',
                                          hover_name='club_name',
                                          title='Posession')

# possession_figure.update_layout({ 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#                                                 'paper_bgcolor': 'rgba(0, 0, 0, 0)',
#                                                 })

possession_figure.show()

# Shots

In [47]:
df_filtered_match_stats['shots_off_target'] = df_filtered_match_stats['shots'] - df_filtered_match_stats['shots_on_target']
df_filtered_match_stats

,match_id,club_id,possession,shots,shots_on_target,touches,passes,tackles,clearances,corners,offsides,fouls_conceded,yellow_cards,red_cards,stadium_id,club_name,shots_off_target
0,59183,163,61.7,12,3,833,610,18,13,5,2,10,2,0,176,Tottenham Hotspur,9
1,59183,158,38.3,10,5,583,371,29,16,5,0,14,1,0,171,Southampton,5


In [48]:
df_filtered_match_shots = df_filtered_match_stats[['club_name', 'shots', 'shots_on_target', 'shots_off_target']].copy()
df_filtered_match_shots.rename(columns={'shots_on_target': 'On Target',
                                     'shots_off_target': 'Off Target',}, inplace=True)
df_filtered_match_shots

,club_name,shots,On Target,Off Target
0,Tottenham Hotspur,12,3,9
1,Southampton,10,5,5


In [49]:
df_filtered_match_shots = df_filtered_match_shots.melt(id_vars=['club_name', 'shots'], var_name='shots_type', value_name='count')
df_filtered_match_shots

,club_name,shots,shots_type,count
0,Tottenham Hotspur,12,On Target,3
1,Southampton,10,On Target,5
2,Tottenham Hotspur,12,Off Target,9
3,Southampton,10,Off Target,5


In [50]:
shots_figure = px.sunburst(df_filtered_match_shots, path=['club_name', 'shots_type', 'count'],
                                          values='count',
                                          labels={'id': 'Club'},
                                          title='Shots')

# shots_figure.update_layout({ 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#                                            'paper_bgcolor': 'rgba(0, 0, 0, 0)',
#                                            })

shots_figure.show()

# Cards

In [51]:
df_filtered_match_cards = df_filtered_match_stats[['club_name', 'yellow_cards', 'red_cards']]
df_filtered_match_cards = df_filtered_match_cards.rename(columns={'yellow_cards': 'Yellow',
                                                            'red_cards': 'Red'})
df_filtered_match_cards

,club_name,Yellow,Red
0,Tottenham Hotspur,2,0
1,Southampton,1,0


In [52]:
    cards_figure = px.bar(df_filtered_match_cards, x='club_name', y=['Yellow', 'Red'],
                                        labels={'variable': 'Card', 'club_name':'Club', 'value': 'Cards Count'})

    # cards_figure.update_layout({ 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #                                            'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    #                                            })

    cards_figure.show()

# Cards

In [53]:
df_filtered_match_passes = df_filtered_match_stats[['club_name', 'passes']]
df_filtered_match_passes

,club_name,passes
0,Tottenham Hotspur,610
1,Southampton,371


In [54]:
passes_figure = px.bar(df_filtered_match_passes, x='club_name', y='passes',
                                    labels={'variable': 'Passes', 'club_name':'Club'},
                                    color='club_name')

passes_figure.update_layout({ 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                             'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                             })

passes_figure.show()

# Line-ups

In [55]:
# Get player stats in target match
df_filtered_match_squad = df_player_stats[df_player_stats['match_id'] == df_filtered_match['match_id'].values[0]]

# Get player names
df_filtered_match_squad = df_filtered_match_squad.merge(df_player, on='player_id')

# Get team of each player 
df_filtered_match_squad['club_name'] = df_filtered_match_squad['is_home_side'].apply(lambda x: home_team_name if x else away_team_name)

# sort player by players in line-up
df_filtered_match_players = df_filtered_match_squad.sort_values(by='is_in_starting_11', ascending=False)

df_filtered_match_players.head()

,player_id,match_id,is_in_starting_11,substitution_on,substitution_off,yellow_card,red_card,is_home_side,player_name,country,club_name
0,p101178,59183,1,NaN,NaN,0,0,0,James Ward-Prowse,England,Southampton
15,p203389,59183,1,NaN,84',0,0,0,Nathan Tella,England,Southampton
38,p93264,59183,1,NaN,NaN,1,0,1,Eric Dier,England,Tottenham Hotspur
37,p93100,59183,1,NaN,NaN,0,0,0,Jannik Vestergaard,Denmark,Southampton
36,p91047,59183,1,NaN,NaN,0,0,0,Stuart Armstrong,Scotland,Southampton


In [56]:
# # Get teams lines-ups and benches
# home_team_line_up_mask = (df_filtered_match_players['is_in_starting_11'] == 1) & (df_filtered_match_players['is_home_side'] == 1)
# home_team_bench_mask = (df_filtered_match_players['is_in_starting_11'] == 0) & (df_filtered_match_players['is_home_side'] == 1)
# away_team_line_up_mask = (df_filtered_match_players['is_in_starting_11'] == 1) & (df_filtered_match_players['is_home_side'] == 0)
# away_team_bench_mask = (df_filtered_match_players['is_in_starting_11'] == 0) & (df_filtered_match_players['is_home_side'] == 0)

# df_home_team_line_up = df_filtered_match_players[home_team_line_up_mask]
# df_away_team_line_up = df_filtered_match_players[away_team_line_up_mask]
# df_home_team_bench = df_filtered_match_players[home_team_bench_mask]
# df_away_team_bench = df_filtered_match_players[away_team_bench_mask]
# df_home_team_line_up.head()

In [57]:
# Get teams squad
home_team_squad = df_filtered_match_players[df_filtered_match_players['is_home_side'] == 1]['player_name']
away_team_squad = df_filtered_match_players[df_filtered_match_players['is_home_side'] == 0]['player_name']

In [58]:
import plotly.graph_objects as go

# create the original table
line_ups_figure = go.Figure(data=[go.Table(
        header=dict(values=[home_team_name, away_team_name],
                    fill_color='paleturquoise',
                    ),
        cells=dict(values=[home_team_squad, away_team_squad],
                fill_color='lavender',
                )),
    ])

# highlight the first 11 rows with a different background color

cell_colors = [['lightblue'] * 11 + ['white'] * (len(home_team_squad)-11)]
cell_colors

line_ups_figure.update_traces(cells=dict(fill_color=cell_colors))

# set the name of the table to the title
line_ups_figure.update_layout(title='Line Ups')

# display the table
line_ups_figure.show()